In [ ]:
!pip install -q -U google-generativeai

# Build

## Mount drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Import Lib + Get API Key

In [ ]:
import google.generativeai as genai
from PIL import Image

In [ ]:
from google.colab import userdata

#Get API KEY from colab.
GOOGLE_API_KEY = userdata.get('GeminiAPI')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924


## Configuration

In [ ]:
model_name = 'gemini-1.5-flash'

In [ ]:
# Define the ClientFactory class to manage API clients
class ClientFactory:
    def __init__(self):
        self.clients = {}

    def register_client(self, name, client_class):
        self.clients[name] = client_class

    def create_client(self, name, **kwargs):
        client_class = self.clients.get(name)
        if client_class:
            return client_class(**kwargs)
        raise ValueError(f"Client '{name}' is not registered.")

In [ ]:
# Register and create the Google generative AI client
client_factory = ClientFactory()
client_factory.register_client('google', genai.GenerativeModel)

client_kwargs = {
    "model_name": model_name,
    "generation_config": {"temperature": 0.4},
    "system_instruction": None,
}

client = client_factory.create_client('google', **client_kwargs)

## Inferences

In [ ]:
# !unzip -q /content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu.zip -d /content/drive/MyDrive/VIVQA4Edu/Dataset

In [ ]:
import os

def load_image(folder_path):
  image_files = []
  for filename in os.listdir(folder_path):
      if filename.endswith(('.jpg', '.jpeg', '.png')):
          image_files.append(os.path.join(folder_path, filename))
  return image_files


In [ ]:
folder_path = '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu'

In [ ]:
image_paths = load_image(folder_path)
image_paths

['/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2363.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2364.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2365.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2366.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2367.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2368.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2369.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2370.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2371.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2372.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2373.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2374.png',
 '/content/drive/MyDrive/VIVQA4Edu/Dataset/ViVQA4Edu/Screenshot_2375.png',
 '/content/drive/MyDrive/

In [ ]:
batch_files_list = []
imageID_list = []
# Define the number of images per batch
num_images_per_batch = 1

# for i in range(0, len(image_paths), num_images_per_batch):
for i in range(600, 800, num_images_per_batch):
    batch_paths = image_paths[i:i + num_images_per_batch]
    batch_files = []
    batch_imageIDs = []
    for path in batch_paths:
      # ID = int(path.split('/')[-1].split('.')[0]) #Use for EVJVQA
      ID = int(os.path.basename(path).split('_')[1].split('.')[0]) #Use for ViVQA4Edu
      batch_imageIDs.append(ID)
      # uploaded_file = genai.upload_file(path=path)
      # batch_files.append(uploaded_file)

      try:
          uploaded_file = genai.upload_file(path=path)
          batch_files.append(uploaded_file)
      except ConnectionError as e:
          print(f"Connection error uploading {path}: {e}")
          print("Retrying in 5 seconds...")
          time.sleep(5)  # Wait for 5 seconds before retrying
          try:
              uploaded_file = genai.upload_file(path=path)
              batch_files.append(uploaded_file)
          except ConnectionError as e:
              print(f"Failed to upload {path} after retry: {e}")
              # You might want to handle this failure differently,
              # like skipping the file or logging the error
              continue # Skip this file and move to the next


    batch_files_list.append(batch_files)
    imageID_list.append(batch_imageIDs)



batch_files_list

[[genai.File({
      'name': 'files/gwj2i84m2xpu',
      'display_name': 'Screenshot_3000.png',
      'mime_type': 'image/png',
      'sha256_hash': 'Y2Y0YjVlOTkwNzdhOGI0Yzc5MTJlMjBlMGQ1YTRhYmMzZDhkYmZhYjQwNTgzNTY5NzIzMGVhZTVlMWRlYTY2ZA==',
      'size_bytes': '401622',
      'state': 'ACTIVE',
      'uri': 'https://generativelanguage.googleapis.com/v1beta/files/gwj2i84m2xpu',
      'create_time': '2024-11-08T15:04:15.651678Z',
      'expiration_time': '2024-11-10T15:04:15.592638867Z',
      'update_time': '2024-11-08T15:04:15.651678Z'})],
 [genai.File({
      'name': 'files/u1j6p0mvdnoi',
      'display_name': 'Screenshot_3001.png',
      'mime_type': 'image/png',
      'sha256_hash': 'NTg0YzNmYzMzZWU5ZDI0NDEwNDgzNWY3MjlhZGVhZWFhODYwYTljMTdiMGQ4MTcxZGZlODdhZjI4NGU5MzAzOQ==',
      'size_bytes': '135899',
      'state': 'ACTIVE',
      'uri': 'https://generativelanguage.googleapis.com/v1beta/files/u1j6p0mvdnoi',
      'create_time': '2024-11-08T15:04:19.180255Z',
      'expiration_time

## Image Captioning

In [ ]:
prompt_cap = """Hãy mô tả chi tiết từng bức ảnh bằng Tiếng Việt. Mỗi mô tả **không dưới 35 từ** và **không quá 70 từ**. Chỉ phản hồi bằng tiếng Việt và phản hồi theo định dạng như sau:
'
Ảnh 1: [Mô tả 1]
Ảnh 2: [Mô tả 2]
Ảnh 3: [Mô tả 3]
Ảnh 4: [Mô tả 4]
Ảnh 5: [Mô tả 5]
...
Ảnh n: [Mô tả n]
'
"""


In [ ]:
caption_batch_list = []

for batch_files in batch_files_list:
  contents = [prompt_cap] + batch_files
  response = client.generate_content(contents=contents, stream=True)
  response.resolve()

  try:
  # Check if 'candidates' list is not empty
    if response.candidates:
        # Access the first candidate's content if available
        if response.candidates[0].content.parts:
            generated_text = response.candidates[0].content.parts[0].text
            caption_batch_list.append(generated_text)
        else:
            print("No generated text found in the candidate.")
            caption_batch_list.append("Không có chú thích")
    else:
        print("No candidates found in the response.")
  except (AttributeError, IndexError) as e:
      print("Error:", e)
      continue

In [ ]:
import re

def extract_descriptions(text):
  """Tách các mô tả từ chuỗi string.

  Args:
    text: Chuỗi string đầu vào.

  Returns:
    Một list các mô tả.
  """
  pattern = r"Ảnh \d+:\s*(.*)"  # Regex cho Image captioning
  descriptions = re.findall(pattern, text)
  return descriptions



In [ ]:
descriptions_list = []
for batch_list in caption_batch_list:
  descriptions_batch = extract_descriptions(batch_list)
  descriptions_list.append(descriptions_batch)

In [ ]:
flat_descrtiptions_list = [item for sublist in descriptions_list for item in sublist]
flat_imageID_list = [item for sublist in imageID_list for item in sublist]

In [ ]:
# prompt: Write caption_list and ImageID_list to CSV file\

import csv

with open('/content/drive/MyDrive/VIVQA4Edu/EVJVQA_Vicaptions.csv', 'w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['ImageID', 'Caption'])  # Write header row
  for i in range(len(flat_imageID_list)):
    writer.writerow([flat_imageID_list[i], flat_descrtiptions_list[i]])

## Visual Question Answering

In [ ]:
prompt_ques = """Bạn là một người chuyên gia trong lĩnh vực gán nhãn dữ liệu. Hãy đặt ra các câu hỏi về nội dung của bức ảnh bằng Tiếng Việt. Chỉ đưa ra các câu hỏi liên quan đến nội dung bức ảnh (Nếu không thể đưa ra câu hỏi thì hãy phản hồi "Không thể tìm ra câu hỏi" cho ảnh tương ứng), chỉ phản hồi bằng tiếng Việt và phản hồi theo định dạng như sau (**Quan trọng! Chú ý phản hồi đúng định dạng**):
'
Ảnh: [Câu hỏi 1,
      Câu hỏi 2,
      ...,
      Câu hỏi k]
'
"""

In [ ]:
# prompt_ques = """Bạn là một người chuyên gia trong lĩnh vực gán nhãn dữ liệu. Hãy đặt ra các câu hỏi về nội dung của từng bức ảnh bằng Tiếng Việt. Chỉ đưa ra các câu hỏi liên quan đến nội dung bức ảnh (Nếu không thể đưa ra câu hỏi thì hãy phản hồi "Không thể tìm ra câu hỏi" cho ảnh tương ứng), chỉ phản hồi bằng tiếng Việt và phản hồi theo định dạng như sau (Quan trọng! Chú ý phản hồi đúng định dạng):
# '
# Ảnh 1: [Câu hỏi 1,
#         Câu hỏi 2,
#         ...
#         Câu hỏi k]
# Ảnh 2: [Câu hỏi 1,
#         Câu hỏi 2,
#         ...
#         Câu hỏi k]
# Ảnh 3: [Câu hỏi 1,
#         Câu hỏi 2,
#         ...
#         Câu hỏi k]

# ...
# Ảnh n: [Câu hỏi 1,
#         Câu hỏi 2,
#         ...
#         Câu hỏi k]
# '
# """

In [ ]:
question_batch_list = []

for batch_files in batch_files_list:
  contents = [prompt_ques] + batch_files
  response = client.generate_content(contents=contents, stream=True)
  response.resolve()

  try:
  # Check if 'candidates' list is not empty
    if response.candidates:
        # Access the first candidate's content if available
        if response.candidates[0].content.parts:
            generated_text = response.candidates[0].content.parts[0].text
            question_batch_list.append(generated_text)
        else:
            print("No generated text found in the candidate.")
            caption_batch_list.append("Không có chú thích")
    else:
        print("No candidates found in the response.")
  except (AttributeError, IndexError) as e:
      print("Error:", e)
      continue


In [ ]:
question_batch_list

["'\nẢnh: Họ đang làm gì?\n      Có bao nhiêu người trong ảnh?\n      Họ đang ở đâu?\n      Họ đang mặc gì?\n'",
 "'\nẢnh: Có bao nhiêu cuốn sách trên kệ?\n      Có bao nhiêu đồ chơi trên kệ?\n      Có bao nhiêu người trong hình?\n      Ai đang cầm sách?\n      Ai đang cầm gấu bông?\n      Ai đang cầm xe hơi?\n      Ai đang cầm robot?\n'",
 "'\nẢnh: Có bao nhiêu bạn nhỏ trong bức tranh?\n      Các bạn nhỏ đang làm gì?\n      Bạn nhỏ nào đang vẽ tranh?\n      Bạn nhỏ nào đang ngồi dưới gốc cây?\n      Có những con vật gì trong bức tranh?\n'",
 "'\nẢnh: Họ đang ở đâu?,\n      Họ đang làm gì?,\n      Có những loại rau củ gì trong ảnh?,\n      Ai đang xách túi nặng hơn?\n'",
 "'\nẢnh: Con vật nào nặng hơn?\n      Con vật nào nhẹ hơn?\n'",
 "'\nẢnh: Bên trái cân có những loại trái cây gì?,\n      Bên phải cân có những loại rau củ gì?,\n      Cân nghiêng về phía nào?\n'",
 "'\nẢnh: Hai bạn nhỏ đang làm gì?\n      Bạn gái đang rót chất lỏng gì?\n      Bạn trai đang làm gì?\n'",
 "'\nẢnh: Hai 

In [ ]:
# imageID_list[4]

In [ ]:
prompt_ans = """Bạn là một người chuyên gia trong lĩnh vực gán nhãn dữ liệu. Bạn được cũng cấp một bộ các câu hỏi đi kèm với hình ảnh tương ứng có định dạng như sau:
'
Ảnh: [Câu hỏi 1,
      Câu hỏi 2,
      ...,
      Câu hỏi k]

'

Dựa vào hình ảnh tương ứng, hãy trả lời các câu hỏi sau bằng Tiếng Việt. Chỉ đưa ra các câu trả lời khi bạn chắc chắn đáp án (Nếu như không rõ đáp án, thì hãy trả lời là 'Tôi không biết đáp án' cho mỗi câu hỏi tương ứng) , chỉ phản hồi bằng tiếng Việt và phản hồi theo định dạng như sau (**Quan trọng! Chú ý phản hồi đúng định dạng**):
'
Ảnh: [Câu trả lời cho câu hỏi 1,
      Câu trả lời cho câu hỏi 2,
        ...
      Câu trả lời cho câu hỏi k]
'
"""

In [ ]:
# prompt_ans = """Bạn là một người chuyên gia trong lĩnh vực gán nhãn dữ liệu. Bạn được cũng cấp một bộ các câu hỏi đi kèm với hình ảnh tương ứng có định dạng như sau:
# '
# Ảnh 1: [Câu hỏi 1,
#         Câu hỏi 2,
#         ...
#         Câu hỏi k]
# Ảnh 2: [Câu hỏi 1,
#         Câu hỏi 2,
#         ...
#         Câu hỏi k]
# Ảnh 3: [Câu hỏi 1,
#         Câu hỏi 2,
#         ...
#         Câu hỏi k]
# ...
# Ảnh n: [Câu hỏi 1,
#         Câu hỏi 2,
#         ...
#         Câu hỏi k]
# '
# Dựa vào hình ảnh tương ứng, hãy trả lời các câu hỏi sau bằng Tiếng Việt. Chỉ đưa ra các câu trả lời khi bạn chắc chắn đáp án (Nếu như không rõ đáp án, thì hãy trả lời là 'Tôi không biết đáp án' cho mỗi câu hỏi tương ứng) , chỉ phản hồi bằng tiếng Việt và phản hồi theo định dạng như sau (Quan trọng! Chú ý phản hồi đúng định dạng):
# Ảnh 1: [Câu trả lời cho câu hỏi 1,
#         Câu trả lời cho câu hỏi 2,
#         ...
#         Câu trả lời cho câu hỏi k]
# Ảnh 2: [Câu trả lời cho câu hỏi 1,
#         Câu trả lời cho câu hỏi 2,
#         ...
#         Câu trả lời cho câu hỏi k]
# Ảnh 3: [Câu trả lời cho câu hỏi 1,
#         Câu trả lời cho câu hỏi 2,
#         ...
#         Câu trả lời cho câu hỏi k]
# ...
# Ảnh n: [Câu trả lời cho câu hỏi 1,
#         Câu trả lời cho câu hỏi 2,
#         ...
#         Câu trả lời cho câu hỏi k]
# """

In [ ]:
answer_batch_list = []

for index, batch_files in enumerate(batch_files_list):
  contents = [prompt_ans] + batch_files + [question_batch_list[index]]
  response = client.generate_content(contents=contents, stream=True)
  response.resolve()

  try:
  # Check if 'candidates' list is not empty
    if response.candidates:
        # Access the first candidate's content if available
        if response.candidates[0].content.parts:
            generated_text = response.candidates[0].content.parts[0].text
            answer_batch_list.append(generated_text)
        else:
            print("No generated text found in the candidate.")
            caption_batch_list.append("Không có chú thích")
    else:
        print("No candidates found in the response.")
  except (AttributeError, IndexError) as e:
      print("Error:", e)
      continue

In [ ]:
len(answer_batch_list)

200

In [ ]:
import re

def extract_question(text):
  # pattern = r"Ảnh \d+:\s*\[(.*?)\]"  # Tìm kiếm các câu hỏi trong dấu ngoặc vuông sau "Ảnh" + số thứ tự cho batch
  pattern = r"Ảnh:\s*(.*)?"
  question_list = []
  for match in re.finditer(pattern, text, re.DOTALL):  # re.DOTALL để khớp với cả newline
      question_str = match.group(1)  # Lấy nội dung trong ngoặc vuông
      question = [q.strip() for q in question_str.split(",\n") if q.strip()]  # Tách các câu hỏi và loại bỏ khoảng trắng thừa
      question_list.append(question)

  return question_list

In [ ]:
import re

def extract_answer(text):
  # pattern = r"Ảnh \d+:\s*\[(.*?)\]"  # Tìm kiếm các câu hỏi trong dấu ngoặc vuông sau "Ảnh" + số thứ tự cho batch
  pattern = r"Ảnh:\s*(.*)"
  answer_list = []
  for match in re.finditer(pattern, text, re.DOTALL):  # re.DOTALL để khớp với cả newline
      answer_str = match.group(1)  # Lấy nội dung trong ngoặc vuông
      answer = [q.strip() for q in answer_str.split(",\n") if q.strip()]  # Tách các câu hỏi và loại bỏ khoảng trắng thừa
      answer_list.append(answer)

  return answer_list


In [ ]:
questions_list = []
answers_list = []
for batch_list in question_batch_list:
  questions_batch = extract_question(batch_list)
  questions_list.append(questions_batch)

for batch_list in answer_batch_list:
  answers_batch = extract_answer(batch_list)
  answers_list.append(answers_batch)

In [ ]:
def find_empty_lists(input_list):
       """Finds all empty list elements within a list.

       Args:
           input_list: The input list to search.

       Returns:
           A list of indices of the empty list elements.
       """
       empty_list_indices = []
       for index, element in enumerate(input_list):
           if isinstance(element, list) and len(element) == 0:  # Check if it's a list and empty
               empty_list_indices.append(index)
       return empty_list_indices



In [ ]:
empty_list = []
empty_list = empty_list + find_empty_lists(answers_list)
empty_list = empty_list + find_empty_lists(questions_list)
empty_list = list(set(empty_list))
empty_list

[91]

In [ ]:
questions_list[91]

[['Bạn trai đang làm gì?',
  'Bạn gái đang làm gì?',
  'Bạn gái đang viết gì trên bảng?',
  "Bạn trai và bạn gái đang ở đâu?\n'"]]

In [ ]:
for index in empty_list:
  questions_list.pop(index)
  answers_list.pop(index)
  imageID_list.pop(index)

In [ ]:
flat_questions_list = [item for sublist in questions_list for item in sublist]
flat_answers_list = [item for sublist in answers_list for item in sublist]
flat_imageID_list = [item for sublist in imageID_list for item in sublist]

In [ ]:
# flat_questions_list[5]

In [ ]:
len(flat_imageID_list)

199

In [ ]:
# prompt: Write caption_list and ImageID_list to CSV file\

import csv

with open('/content/drive/MyDrive/VIVQA4Edu/VIVQA4Edu_Vicaptions_4.csv', 'w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['ImageID', 'Question', 'Answer'])  # Write header row
  for i in range(len(flat_imageID_list)):
    writer.writerow([flat_imageID_list[i], flat_questions_list[i], flat_answers_list[i]])
